In [30]:
import random
import numpy as np
import mysql.connector
from mysql.connector import Error

connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="vachas289",
    database="School"
)
if connection.is_connected():
    print("Connected to MySQL server")
    cursor = connection.cursor()

    #droping the tables which are wrongly created first
    cursor.execute("DROP TABLE IF EXISTS Student_Demographics, Student_Information, School_Information")
    #Create the tables with primary keys
    cursor.execute("""
    CREATE TABLE School_Information (
        School_ID INT,
        School_Name VARCHAR(255),
        Class INT,
        Monthly_Fee INT,
        PRIMARY KEY (School_ID, Class))""")

    cursor.execute("""
    CREATE TABLE Student_Information (
        Student_ID INT,
        Class INT,
        School_ID INT,
        Overall_Marks INT,
        PRIMARY KEY (Student_ID),
        FOREIGN KEY (School_ID, Class) REFERENCES School_Information(School_ID, Class))""")

    cursor.execute("""
    CREATE TABLE Student_Demographics (
        Student_ID INT,
        Native VARCHAR(50),
        Age INT,
        Monthly_Family_Income INT,
        PRIMARY KEY (Student_ID),
        FOREIGN KEY (Student_ID) REFERENCES Student_Information(Student_ID))""")

    print("Tables created successfully")

    #data for school_information
    school_names = ['ABN', 'Montfort', 'RLA']
    schools_data = []

    class_fees = {}
    school_ids = {
        'ABN': 1,
        'Montfort': 2,
        'RLA': 3
    }

    #school_information monthly fees and name based on classes and category
    for school_name in school_names:
        school_id = school_ids[school_name]
        if school_name == 'ABN':
            classes = range(1, 9)
        else:
            classes = range(9, 13)
        
        # Generate random monthly fee for each class in the school
        class_fees[school_name] = {class_num: random.randint(1000, 5000) for class_num in classes}
        
        for class_num, fee in class_fees[school_name].items():
            schools_data.append((school_id, school_name, class_num, fee))

    #inserting school_information
    cursor.executemany("""
    INSERT INTO School_Information (School_ID, School_Name, Class, Monthly_Fee)
    VALUES (%s, %s, %s, %s)""", schools_data)

    print("School Information data inserted successfully")

    #data of student_information
    students_data = []
    student_ids = []

    #Total number of students per class
    num_students_per_class = 50
    primary_school_classes = 8
    secondary_school_classes = 4

    #primary School (Class 1 to 8)
    for class_num in range(1, primary_school_classes + 1):
        school_id = 1
        for student_num in range(num_students_per_class):
            student_id = len(student_ids) + 1
            student_ids.append(student_id)

            #marks
            marks = int(np.random.normal(350, 100))
            marks = max(1, min(500, marks))

            students_data.append((student_id, class_num, school_id, marks))

    #Secondary Schools (Class 9 to 12)
    for school_index, school_name in enumerate(['Montfort', 'RLA'], start=2):
        school_id = school_index
        for class_num in range(9, 13):
            for student_num in range(num_students_per_class):
                student_id = len(student_ids) + 1
                student_ids.append(student_id)

                marks = int(np.random.normal(350, 100))
                marks = max(1, min(500, marks))

                students_data.append((student_id, class_num, school_id, marks))

    #inserting data for student_information
    cursor.executemany("""
    INSERT INTO Student_Information (Student_ID, Class, School_ID, Overall_Marks)
    VALUES (%s, %s, %s, %s)""", students_data)

    print("Student Information data inserted successfully")

    #Student_Demographics
    student_demographics_data = []

    #demographics data for each student using the student id from df student_id for matching the student_id and also the age is dependent on the class
    for student_id in student_ids:
        native = np.random.choice(['Indian', 'Foreigner'],p=[0.9,0.1])
        age = int(np.random.normal(student_id % 12 + 6, 2))
        age = max(6, age)
        monthly_family_income = int(np.random.normal(25000, 10000))
        monthly_family_income = max(1000, monthly_family_income)

        student_demographics_data.append((student_id, native, age, monthly_family_income))

    #inserting the student_demographics in MySQL
    cursor.executemany("""
    INSERT INTO Student_Demographics (Student_ID, Native, Age, Monthly_Family_Income)
    VALUES (%s, %s, %s, %s)""", student_demographics_data)

    print("Student Demographics data inserted successfully")

    connection.commit()
    cursor.close()
    connection.close()
    print("MySQL connection is closed")


Connected to MySQL server
Tables created successfully
School Information data inserted successfully
Student Information data inserted successfully
Student Demographics data inserted successfully
MySQL connection is closed


In [31]:
import pandas as pd
import mysql.connector

connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="vachas289",
    database="School"
)
if connection.is_connected():
    print("Connected to MySQL server")
    
    query_school_information = "SELECT * FROM School_Information"
    query_student_information = "SELECT * FROM Student_Information"
    query_student_demographics = "SELECT * FROM Student_Demographics"
    
    school_information_df = pd.read_sql(query_school_information, connection)
    student_information_df = pd.read_sql(query_student_information, connection)
    student_demographics_df = pd.read_sql(query_student_demographics, connection)

    with pd.ExcelWriter("school_data.xlsx") as writer:
        school_information_df.to_excel(writer, sheet_name="School_Information", index=False)
        student_information_df.to_excel(writer, sheet_name="Student_Information", index=False)
        student_demographics_df.to_excel(writer, sheet_name="Student_Demographics", index=False)

    print("Excel file created successfully")

    connection.close()
    print("MySQL connection is closed")


Connected to MySQL server


C:\Users\al6436\AppData\Local\Temp\ipykernel_2744\4092437477.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  school_information_df = pd.read_sql(query_school_information, connection)
C:\Users\al6436\AppData\Local\Temp\ipykernel_2744\4092437477.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  student_information_df = pd.read_sql(query_student_information, connection)
C:\Users\al6436\AppData\Local\Temp\ipykernel_2744\4092437477.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  student_demographics_df = pd.read_sql

Excel file created successfully
MySQL connection is closed


In [32]:
import pandas as pd

# Load the Excel file
file_path = "C:\\Users\\al6436\\Desktop\\school_data.xlsx"
xls = pd.ExcelFile(file_path)

# Load each sheet into a DataFrame
school_information_df = pd.read_excel(xls, 'School_Information')
student_information_df = pd.read_excel(xls, 'Student_Information')
student_demographics_df = pd.read_excel(xls, 'Student_Demographics')

# Display the first few rows of each DataFrame
print("School Information:")
print(school_information_df.head())

print("\nStudent Information:")
print(student_information_df.head())

print("\nStudent Demographics:")
print(student_demographics_df.head())


School Information:
   School_ID School_Name  Class  Monthly_Fee
0          1         ABN      1         3993
1          1         ABN      2         3479
2          1         ABN      3         3446
3          1         ABN      4         3040
4          1         ABN      5         1314

Student Information:
   Student_ID  Class  School_ID  Overall_Marks
0           1      1          1            352
1           2      1          1            412
2           3      1          1            333
3           4      1          1            416
4           5      1          1            262

Student Demographics:
   Student_ID  Native  Age  Monthly_Family_Income
0           1  Indian    7                  26349
1           2  Indian    8                  27593
2           3  Indian    8                  15887
3           4  Indian    9                   9940
4           5  Indian   13                  23618
